In [81]:
# import os

# from typing import List, Dict

# from pydantic import BaseModel, Field
# from dotenv import load_dotenv

# import google.generativeai as genai

# # from crewai import Agent, Task, Crew, LLM
# # from crewai.tools import tool

# # import agentops

# # from tavily import TavilyClient

In [82]:
# load_dotenv("../src/.env")

In [83]:
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [65]:
# import os
# import fitz  # PyMuPDF
# import docx

# def extract_text_from_pdf(file_path):
#     text = ""
#     with fitz.open(file_path) as doc:
#         for page in doc:
#             text += page.get_text()
#     return text

# def extract_text_from_docx(file_path):
#     doc = docx.Document(file_path)
#     return "\n".join([p.text for p in doc.paragraphs])

# def extract_text_from_txt(file_path):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         return f.read()

# def parse_cv(file_path):
#     if file_path.endswith(".pdf"):
#         return extract_text_from_pdf(file_path)
#     elif file_path.endswith(".docx"):
#         return extract_text_from_docx(file_path)
#     elif file_path.endswith(".txt"):
#         return extract_text_from_txt(file_path)
#     else:
#         return None

# # Example: Parse all CVs in a folder
# folder_path = "../data/"
# parsed_cvs = []

# for file_name in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, file_name)
#     text = parse_cv(file_path)
#     if text:
#         parsed_cvs.append({"filename": file_name, "text": text})


# # for cv in parsed_cvs:
# #     print(f"\n--- {cv['filename']} ---")
# #     print(cv['text'])  # print only first 1000 characters to keep it short


# def chunk_text(text, max_words=200):
#     words = text.split()
#     return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]





In [66]:
# import chromadb
# from sentence_transformers import SentenceTransformer

# client = chromadb.PersistentClient(path="./chroma_db")
# collection = client.get_or_create_collection(name="cv_chunks")

# model = SentenceTransformer('all-MiniLM-L6-v2')

# for file in os.listdir("../data/"):
#     path = os.path.join("../data/", file)
#     text = parse_cv(path)
#     if text:
#         chunks = chunk_text(text)
#         embeddings = model.encode(chunks).tolist()
#         for i, chunk in enumerate(chunks):
#             collection.add(
#                 documents=[chunk],
#                 embeddings=[embeddings[i]],
#                 metadatas=[{"filename": file, "chunk": i}],
#                 ids=[f"{file}_{i}"]
#             )
            

In [67]:
# def get_context_from_chromadb(question, collection, top_k=5):
#     results = collection.query(query_texts=[question], n_results=top_k)
#     return "\n".join(results['documents'][0])

# from crewai import Agent, Task
# from pydantic import BaseModel, Field

# class AnswerOutput(BaseModel):
#     answer: str = Field(..., title="Answer to the user's question")

# research_Agent= Agent(
#         role="Helpful Research Assistant",
#         goal="Answer user questions using provided context only.",
#         backstory="You are a helpful assistant that answers questions from CV/resume chunks.",
#         llm=basic_llm,
#         verbose=True,
#     )

# research_Agent_task = Task(
#     description="\n".join([
#         f"you are an expert in a {user_input} field to help beginner researchers in their writings .",
#         "Provide a list of 3 to 5 trending topics or articals with a brief description for each.",
#         "Focus on recent research interests supported by publication trends.",
#         "Output in JSON format with 'topics' as list of objects {name, description}."
#     ]),
#     expected_output="JSON object with list of trending topics and descriptions.",
#     output_json=TrendingTopicsOutput,
#     output_file=os.path.join(output_dir, "step_1_trending_topics.json"),
#     agent=trending_topics_agent,
# )

# def build_answer_task(user_question, collection, output_dir, llm):
#     context = get_context_from_chromadb(user_question, collection)
#     agent = build_answer_agent(llm)

#     return Task(
#         description="\n".join([
#             f"Context:\n{context}",
#             f"Question: {user_question}",
#             "Answer the question strictly based on the given context.",
#             "Give a clear and concise answer."
#         ]),
#         expected_output="A short and accurate answer.",
#         output_json=AnswerOutput,
#         output_file=os.path.join(output_dir, "qa_answer.json"),
#         agent=agent,
#     )


# question = "What programming languages does the candidate know?"
# task = build_answer_task(question, collection, output_dir="./outputs", llm=basic_llm)
# result = task.execute()
# print(result)


## `01` Import Lib:

In [85]:
import os
from PyPDF2 import PdfReader
import google.generativeai as genai
from dotenv import load_dotenv

## `02` Directories:

In [86]:
input_folder = "../data/"
output_folder = "../cvs_md_outputs"
os.makedirs(output_folder, exist_ok=True)

## `03` Model Config:

In [87]:
load_dotenv("../src/.env")

True

In [88]:
# Configure Gemini
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

## `04` Extract text from a PDF:

In [89]:
def extract_text(pdf_path):
    reader = PdfReader(pdf_path)
    return "\n".join(page.extract_text() for page in reader.pages)

## `05` Convert to markdown:

In [90]:
def convert_to_markdown(text):
    prompt = "\n".join([
        "You are a smart recruiter and CV expert.",

        "Your task is to convert the following CV (extracted as raw PDF text) into clean, structured **Markdown** format using proper headings and bullet points.",

        "Make the Markdown professional, readable, and consistently formatted.",

        "Use the following structure **only if the information is present**.if not specified do not add it. Do NOT guess or add anything that's not in the text.",

        "Add logic to group and classify content based on context and keywords.",

        "Pay special attention to education and trainings:\n"
        "- Include ALL formal degrees and full-time academic programs (e.g., Bachelor's, ITI) under '## Education' Do not add ITI to Internships.\n"
        "- Do NOT mistakenly place formal education in '## Internships & Trainings'.\n"
        "- Short-term workshops, technical training, and certifications belong under '## Internships & Trainings'.",

        "Use this Markdown structure:",

        "# Full Name\n"
        "Job Title (e.g., AI & ML Engineer)\n",

        "## Contact Information\n"
        "* Email: ...\n"
        "* LinkedIn: ...\n"
        "* GitHub: ...\n"
        "* Kaggle: ...\n"
        "* Phone: ...\n"
        "* Address: ...\n"
        "* Birthdate: ...\n"
        "* Marital Status: ...\n",

        "## Profile\n"
        "A short summary paragraph of key strengths, technical areas, and goals.\n",

        "## Education\n"
        "* Degree – University – Date range\n"
        "* GPA, Rank, or Awards if available\n"
        "* Include ITI and all full-time formal education programs\n",

        "## Internships & Trainings\n"
        "* Program or Course – Institution – Date\n"
        "* Short description (1–2 lines) of practical work or skills learned\n"
        "* Include technical trainings, certifications, workshops, and hands-on practice\n",

        "## Projects\n"
        "* **Project Title:** Description, tools used, purpose, and outcomes\n"
        "* If a link is needed but missing, write [link needed]\n",

        "## Professional Experience\n"
        "* Job Title – Company – Date range\n"
        "* List responsibilities and technologies used\n",

        "## Technical Skills\n"
        "* **Programming Languages:** C++, Python, etc.\n"
        "* **Software & Tools:** Git, LabView, MATLAB, ROS, etc.\n"
        "* **Libraries & Frameworks:** TensorFlow, PyTorch, OpenCV, etc.\n"
        "* **Domain Expertise:** Control Systems, Robotics, CNC, SolidWorks, etc.\n"
        "* **Operating Systems:** Linux, Windows, etc.\n"
        "* **Databases:** MySQL, MongoDB, etc.\n",

        "## Personal Skills\n"
        "* Soft skills, mindset, leadership, communication, etc.\n",

        "## Languages\n"
        "* Language: Proficiency (e.g., English: B2)\n",

        "## Courses\n"
        "* Course Title – Platform – Date\n",

        "## Publications (if any)\n"
        "* Title – Journal/Conference – Year\n",

        "## Awards (if any)\n"
        "* Award Name – Granting Body – Year or Description\n",

        "## Activities (if any)\n"
        "* Role or Activity – Organization – Date\n",
        "* Description of involvement or achievements\n",

        "## Volunteering (if any)\n"
        "* Role – Organization – Duration\n"
        "* Description of contributions\n",

        "### Skill Grouping Intuition:\n"
        "- If the skill reflects what the person *uses* (e.g., software, tools), put it under **Skills**.\n"
        "- If the skill reflects *how the person behaves or works* (e.g., problem-solving, leadership), put it under **Personal Skills**.\n"
        "- If unsure, ask: 'Is it a technical tool or a personal trait?'\n",

        "**Clean-up Rules:**\n"
        "- Remove broken lines, random symbols, page numbers, and noise from PDF\n"
        "- Keep spacing and indentation consistent\n"
        "- Never invent or hallucinate content\n",

        "Now convert the following CV into structured Markdown:\n\n" + text
    ])
    
    response = model.generate_content(prompt)
    return response.text


## `06` Processing Files:

In [ ]:
# Process each PDF
for filename in os.listdir(input_folder):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(input_folder, filename)
        print(f"Processing {filename}...")

        text = extract_text(pdf_path)
        markdown = convert_to_markdown(text)

        md_filename = os.path.splitext(filename)[0] + ".md"
        md_path = os.path.join(output_folder, md_filename)

        with open(md_path, "w", encoding="utf-8") as f:
            f.write(markdown)

print("✅ All CVs converted to Markdown.")

Processing Alaa-Hussien-CV.pdf...
Processing Adham Assy - Resume - AI & ML Engineer.pdf...
